In [6]:
# Cell 1: importy i konfiguracja ścieżki do modułów projektu
import json
from pathlib import Path
import sys

import yaml  # type: ignore

# Upewniamy się, że katalog "src" (z modułami projektu) jest na ścieżce importów
src_path = Path("src").resolve()
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from axiomatic_kernel import (
    AxiomKernel,
    VariableSchema,
    AxiomDefinition,
    DecisionLogger,
)
from nl_rule_parser import build_axiom_from_nl, RuleParseError  # noqa: F401
from explanation_engine import DecisionExplainer, ExplanationConfig
from rules_io import load_ruleset_from_file  # noqa: F401
from ruleset_manager import RulesetRegistry, Environment
from rule_analytics import RuleAnalyticsEngine


In [ ]:
# Cell 2: konfiguracja schematu, kernela i wczytanie rulesetu FRAUD z governance (FAZA 2.2)

# === 1) SCHEMA DOPASOWANA DO FRAUD_RULES (tylko typy obsługiwane przez kernel) ===
schema = [
    VariableSchema("amount", "int", "Kwota transakcji w jednostkach minimalnych."),
    VariableSchema("tx_count_24h", "int", "Liczba transakcji w ostatnich 24h."),
    VariableSchema("is_pep", "bool", "Czy klient jest PEP."),
    VariableSchema("is_suspicious", "bool", "Czy transakcja jest podejrzana."),
]

# Katalog na logi decyzji
logs_dir = Path("logs")
logs_dir.mkdir(exist_ok=True)

logger = DecisionLogger(logs_dir / "fraud_rules_demo.jsonl")

kernel = AxiomKernel(
    schema=schema,
    decision_variable="is_suspicious",
    logger=logger,
    rule_version="fraud_rules_v1",  # zostanie nadpisane przez RulesetRegistry
)

# === 2) Wczytanie i rejestracja rulesetu FRAUD w środowisku DEV ===
rules_dir = Path("rules")
rules_dir.mkdir(exist_ok=True)

fraud_rules_path = rules_dir / "fraud_rules_v1.yaml"

# Zakładamy, że plik fraud_rules_v1.yaml istnieje w katalogu "rules".
# Jeśli chcesz, możesz tu dodać kod, który go stworzy przy pierwszym uruchomieniu.

registry = RulesetRegistry()

record = registry.register_ruleset(
    ruleset_id="fraud_rules_v1",
    path=fraud_rules_path,
    environment=Environment.DEV,
)

print(
    f"📘 Zarejestrowano ruleset: {record.key.ruleset_id} "
    f"(v{record.version}) w środowisku {record.key.environment.value}"
)
print(f"Plik: {record.file_path}")

# === 3) Nałożenie rulesetu na kernel przez RulesetRegistry ===
summary = registry.apply_ruleset_to_kernel(
    ruleset_id="fraud_rules_v1",
    environment=Environment.DEV,
    kernel=kernel,
    schema=schema,
    decision_field_fallback="is_suspicious",
    strict=True,
    extra_metadata={"domain": "fraud-demo"},
)

print("\n📊 Podsumowanie ładowania reguł:")
print(f"- total_rules:   {summary.total_rules}")
print(f"- enabled_rules: {summary.enabled_rules}")
print(f"- loaded_rules:  {summary.loaded_rules}")
print(f"- skipped_rules: {summary.skipped_rules}")
print(f"- errors:        {summary.errors}")


In [ ]:
# Cell 3: przykładowe case'y FRAUD (FLAGGED / CLEAN) + wyjaśnienia

explainer = DecisionExplainer(ExplanationConfig(language="pl"))

# Przypadek "podejrzany" – przykład, który wg rulesetu powinien być oflagowany
case_flagged = {
    "amount": 15_000,
    "tx_count_24h": 20,
    "is_pep": True,
}

bundle_flagged = kernel.evaluate(case_flagged)

print("=== RAW BUNDLE (FLAGGED) ===")
print(json.dumps(bundle_flagged, indent=2, ensure_ascii=False))

print("\n=== WYJAŚNIENIE (FLAGGED) ===")
print(explainer.explain(bundle_flagged).to_text(language="pl"))

# Przypadek "czysty" – przykład, który wg rulesetu powinien być CLEAN
case_clean = {
    "amount": 500,
    "tx_count_24h": 1,
    "is_pep": False,
}

bundle_clean = kernel.evaluate(case_clean)

print("\n=== RAW BUNDLE (CLEAN) ===")
print(json.dumps(bundle_clean, indent=2, ensure_ascii=False))

print("\n=== WYJAŚNIENIE (CLEAN) ===")
print(explainer.explain(bundle_clean).to_text(language="pl"))


In [11]:
# Cell 4: demo UNSAT – sprzeczne reguły na osobnym kernelu (niezależne od FRAUD)

from z3 import Implies  # type: ignore

unsat_schema = [
    VariableSchema("amount", "int", "Kwota transakcji (demo UNSAT)."),
    VariableSchema("risk_score", "int", "Pole demo – nieużywane w regułach."),
    VariableSchema("flag", "bool", "Decyzja testowa (demo UNSAT)."),
]

unsat_kernel = AxiomKernel(
    schema=unsat_schema,
    decision_variable="flag",
    logger=None,
    rule_version="demo_unsat_v1",
)


def rule_flag_true(vars_z3):
    amount = vars_z3["amount"]
    flag = vars_z3["flag"]
    return Implies(amount > 10_000, flag == True)


def rule_flag_false(vars_z3):
    amount = vars_z3["amount"]
    flag = vars_z3["flag"]
    return Implies(amount > 10_000, flag == False)


unsat_kernel.add_axiom(
    AxiomDefinition(
        id="amount_flag_true",
        description="If amount > 10000 then flag must be True.",
        build_constraint=rule_flag_true,
    )
)
unsat_kernel.add_axiom(
    AxiomDefinition(
        id="amount_flag_false",
        description="If amount > 10000 then flag must be False.",
        build_constraint=rule_flag_false,
    )
)

case_conflict = {"amount": 15_000, "risk_score": 5}
bundle_unsat = unsat_kernel.evaluate(case_conflict)

print("=== RAW BUNDLE (UNSAT) ===")
print(json.dumps(bundle_unsat, indent=2, ensure_ascii=False))

print("\n=== WYJAŚNIENIE (UNSAT) ===")
print(explainer.explain(bundle_unsat).to_text(language="pl"))


=== RAW BUNDLE (UNSAT) ===
{
  "decision_status": "UNSAT",
  "decision": "ERROR",
  "facts": {
    "amount": 15000,
    "risk_score": 5
  },
  "model": {},
  "satisfied_axioms": [],
  "violated_axioms": [],
  "active_axioms": [],
  "inactive_actions": [],
  "conflicting_axioms": [
    "amount_flag_false",
    "amount_flag_true"
  ],
  "rule_version": "demo_unsat_v1",
  "error": "Constraints are unsatisfiable for given case."
}

=== WYJAŚNIENIE (UNSAT) ===
Decyzja niemożliwa: zestaw reguł jest SPRZECZNY dla tego przypadku (UNSAT). Kluczowe dane wejściowe: amount=15000, risk_score=5.

Konflikty reguł:
- Konflikt między regułami: amount_flag_false, amount_flag_true.

Błąd techniczny: Constraints are unsatisfiable for given case.


In [12]:
# Cell 5: przygotowanie przykładowego pliku CSV z transakcjami

import csv

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

input_path = data_dir / "transactions_demo.csv"

rows = [
    # transaction_id, customer_id, amount, tx_count_24h, is_pep
    ("T001", "C001", 15_000, 1, False),   # wysoka kwota, normalny klient
    ("T002", "C002", 500,    1, False),   # mała kwota, mało transakcji (CLEAN)
    ("T003", "C003", 500,    10, False),  # velocity (dużo transakcji)
    ("T004", "C004", 3_000,  2, True),    # PEP + średnia kwota
    ("T005", "C005", 20_000, 8, True),    # PEP + bardzo wysoka kwota + velocity
    ("T006", "C006", 8_000,  0, False),   # średnia kwota, brak velocity
    ("T007", "C007", 2_500,  6, False),   # tylko velocity
    ("T008", "C008", 1_000,  7, True),    # PEP, ale bardzo mała kwota
]

with input_path.open("w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["transaction_id", "customer_id", "amount", "tx_count_24h", "is_pep"])
    writer.writerows(rows)

print(f"Zapisano przykładowy plik CSV z transakcjami: {input_path}")


Zapisano przykładowy plik CSV z transakcjami: data/transactions_demo.csv


In [13]:
# Cell 5: Rule Analytics – analiza logów FRAUD (FAZA 4, opcjonalnie)
import csv

input_path = data_dir / "transactions_demo.csv"
output_path = data_dir / "transactions_with_explanations.csv"

explainer = DecisionExplainer(ExplanationConfig(language="pl"))

print(f"Czytam dane z: {input_path}")

output_rows = []

with input_path.open("r", newline="", encoding="utf-8") as f_in:
    reader = csv.DictReader(f_in)
    for row in reader:
        # Budujemy case zgodnie ze schema FRAUD:
        #   amount: int
        #   tx_count_24h: int
        #   is_pep: bool
        case = {
            "amount": int(row["amount"]),
            "tx_count_24h": int(row["tx_count_24h"]),
            "is_pep": str(row["is_pep"]).lower() in {"true", "1", "yes", "y", "t"},
        }

        bundle = kernel.evaluate(case)
        logger.log(bundle)  # zapis do logs/fraud_rules_demo.jsonl

        decision = bundle.get("decision")
        status = bundle.get("decision_status")

        # Lista aktywnych reguł (tych, które faktycznie "odpaliły" w tej decyzji)
        active_rules = [ax["id"] for ax in bundle.get("active_axioms", [])]

        # Pełne wyjaśnienie tekstowe po polsku dla tej konkretnej transakcji
        explanation_text = explainer.explain(bundle).to_text(language="pl")

        output_rows.append(
            {
                "transaction_id": row["transaction_id"],
                "customer_id": row["customer_id"],
                "amount": row["amount"],
                "tx_count_24h": row["tx_count_24h"],
                "is_pep": row["is_pep"],
                "decision": decision,
                "decision_status": status,
                "active_rules": ",".join(active_rules),
                "explanation_pl": explanation_text,
            }
        )

# Zapisujemy wynikowy plik CSV z decyzjami i pełnym wyjaśnieniem
with output_path.open("w", newline="", encoding="utf-8") as f_out:
    fieldnames = [
        "transaction_id",
        "customer_id",
        "amount",
        "tx_count_24h",
        "is_pep",
        "decision",
        "decision_status",
        "active_rules",
        "explanation_pl",
    ]
    writer = csv.DictWriter(f_out, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(output_rows)

print(f"\n✅ Zapisano wynikowy plik z decyzjami i wyjaśnieniami: {output_path}")

print("\nPodgląd pierwszych 2 wierszy wynikowych:")
for row in output_rows[:2]:
    print("\n---------------------------")
    print(f"TX {row['transaction_id']} | decyzja: {row['decision']} ({row['decision_status']})")
    print("Aktywne reguły:", row["active_rules"] or "–")
    print("Wyjaśnienie:")
    print(row["explanation_pl"])


Czytam dane z: data/transactions_demo.csv

✅ Zapisano wynikowy plik z decyzjami i wyjaśnieniami: data/transactions_with_explanations.csv

Podgląd pierwszych 2 wierszy wynikowych:

---------------------------
TX T001 | decyzja: FLAGGED (SAT)
Aktywne reguły: fraud.high_amount
Wyjaśnienie:
Decyzja: transakcja została OFLAGOWANA (FLAGGED). Kluczowe dane wejściowe: amount=15000, is_pep=False, tx_count_24h=1.

Powody (aktywne reguły):
- Reguła 'fraud.high_amount': IF amount > 10000 THEN is_suspicious = TRUE

Reguły, które nie zadziałały w tym przypadku:
- Reguła 'fraud.velocity' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: IF tx_count_24h > 5 THEN is_suspicious = TRUE
- Reguła 'fraud.pep_high_risk' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: If is_pep == true and amount > 2000 then is_suspicious = true

---------------------------
TX T002 | decyzja: CLEAN (SAT)
Aktywne reguły: –
Wyjaśnienie:
Decyzja: transakcja została OZNACZONA jako CZYST

In [14]:
analytics_engine = RuleAnalyticsEngine()

analytics_result = analytics_engine.analyze_log_file(
    log_path=logs_dir / "fraud_rules_demo.jsonl",
    ruleset_path=fraud_rules_path,
)

report = analytics_result.as_dict()

print("=== STATYSTYKI DECYZJI ===")
print(json.dumps(report["outcome_stats"], indent=2, ensure_ascii=False))

print("\n=== STATYSTYKI REGUŁ ===")
for rule_id, stats in sorted(report["rule_stats"].items()):
    print(f"\nReguła: {rule_id}")
    print(json.dumps(stats, indent=2, ensure_ascii=False))

coverage = report.get("coverage_report")
if coverage is not None:
    print("\n=== POKRYCIE RULESETU ===")
    print(json.dumps(coverage, indent=2, ensure_ascii=False))


=== STATYSTYKI DECYZJI ===
{
  "total_decisions": 53,
  "by_decision": {
    "FLAGGED": 37,
    "CLEAN": 16
  },
  "by_status": {
    "SAT": 53
  },
  "by_rule_version": {
    "fraud_rules_v1": 4,
    "fraud_rules_v1:1.0.0@DEV": 49
  },
  "unsat_cases": 0,
  "error_cases": 0
}

=== STATYSTYKI REGUŁ ===

Reguła: fraud.high_amount
{
  "rule_id": "fraud.high_amount",
  "description": "IF amount > 10000 THEN is_suspicious = TRUE",
  "total_occurrences": 53,
  "satisfied": 53,
  "violated": 0,
  "active": 19,
  "inactive": 34,
  "in_conflict": 0
}

Reguła: fraud.pep_high_risk
{
  "rule_id": "fraud.pep_high_risk",
  "description": "If is_pep == true and amount > 2000 then is_suspicious = true",
  "total_occurrences": 39,
  "satisfied": 39,
  "violated": 0,
  "active": 11,
  "inactive": 28,
  "in_conflict": 0
}

Reguła: fraud.velocity
{
  "rule_id": "fraud.velocity",
  "description": "IF tx_count_24h > 5 THEN is_suspicious = TRUE",
  "total_occurrences": 53,
  "satisfied": 53,
  "violated": 0

In [16]:
# Cell X: Raport AML na podstawie pliku CSV z wyjaśnieniami
# oraz zapis tego raportu do pliku TXT

import csv

# Upewniamy się, że mamy katalog z danymi
if "data_dir" not in globals():
    data_dir = Path("data")

csv_path = data_dir / "transactions_with_explanations.csv"
report_path = data_dir / "aml_report.txt"

print(f"📄 Wczytuję raport z: {csv_path}")

# Ten string będziemy wypełniać treścią raportu, a na końcu zapiszemy do pliku TXT
report_text = ""

if not csv_path.exists():
    msg = "❌ Plik transactions_with_explanations.csv nie istnieje. Najpierw uruchom komórkę z batchem CSV."
    print(msg)
    report_text += msg + "\n"
else:
    with csv_path.open("r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        rows = list(reader)

    if not rows:
        msg = "Plik jest pusty, brak transakcji do raportu."
        print(msg)
        report_text += msg + "\n"
    else:
        header = f"=== RAPORT AML – {len(rows)} transakcji ===\n\n"
        print(header)
        report_text += header

        for idx, row in enumerate(rows, start=1):
            # Parsowanie pól
            try:
                amount = int(row["amount"])
            except Exception:
                amount = row["amount"]

            try:
                tx_count_24h = int(row["tx_count_24h"])
            except Exception:
                tx_count_24h = row["tx_count_24h"]

            is_pep_raw = str(row.get("is_pep", "")).strip()
            is_pep_bool = is_pep_raw.lower() in {"true", "1", "yes", "y", "t"}

            decision = row.get("decision", "")
            status = row.get("decision_status", "")
            active_rules = row.get("active_rules") or ""
            explanation_text = row.get("explanation_pl", "").strip()

            # Formatowanie bloku raportu
            block = []
            block.append("=" * 70)
            block.append(f"📌 Transakcja {row.get('transaction_id', 'N/A')} | Klient: {row.get('customer_id', 'N/A')}")
            block.append(
                f"Kwota: {amount} | Liczba transakcji 24h: {tx_count_24h} | "
                f"PEP: {is_pep_bool} (raw: {is_pep_raw})"
            )
            block.append(f"Decyzja silnika: {decision} (status: {status})")

            if active_rules:
                block.append(f"Aktywne reguły: {active_rules}")
            else:
                block.append("Aktywne reguły: brak – żadna reguła nie została uruchomiona (case CLEAN).")

            block.append("\nUzasadnienie (pełne wyjaśnienie):")
            block.append(explanation_text or "(brak wyjaśnienia w pliku)")
            block.append("")  # pusta linia

            block_text = "\n".join(block)
            print(block_text)

            report_text += block_text + "\n"

        footer = "\n" + "=" * 70 + "\n🏁 Koniec raportu AML.\n"
        print(footer)
        report_text += footer

# Zapis do pliku TXT
with report_path.open("w", encoding="utf-8") as f:
    f.write(report_text)

print(f"\n💾 Zapisano raport AML do pliku: {report_path}")


📄 Wczytuję raport z: data/transactions_with_explanations.csv
=== RAPORT AML – 8 transakcji ===


📌 Transakcja T001 | Klient: C001
Kwota: 15000 | Liczba transakcji 24h: 1 | PEP: False (raw: False)
Decyzja silnika: FLAGGED (status: SAT)
Aktywne reguły: fraud.high_amount

Uzasadnienie (pełne wyjaśnienie):
Decyzja: transakcja została OFLAGOWANA (FLAGGED). Kluczowe dane wejściowe: amount=15000, is_pep=False, tx_count_24h=1.

Powody (aktywne reguły):
- Reguła 'fraud.high_amount': IF amount > 10000 THEN is_suspicious = TRUE

Reguły, które nie zadziałały w tym przypadku:
- Reguła 'fraud.velocity' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: IF tx_count_24h > 5 THEN is_suspicious = TRUE
- Reguła 'fraud.pep_high_risk' była spełniona logicznie, ale jej warunek nie dotyczył tego przypadku: If is_pep == true and amount > 2000 then is_suspicious = true

📌 Transakcja T002 | Klient: C002
Kwota: 500 | Liczba transakcji 24h: 1 | PEP: False (raw: False)
Decyzja silnika: CLEAN (s